In [ ]:
!conda install ephem --yes

In [1]:
import ephem
import numpy
import pandas
from SciServer import SciDrive
from astropy import units
from astropy.coordinates import SkyCoord
print('Done')

Done


In [2]:
#SciDrive.directoryList('mpc')
mpc = SciDrive.download('mpc/MPCORB.DAT', localFilePath='mpcorb.dat')
print('Done')

Done


In [3]:
df = pandas.read_fwf('mpcorb.dat')
df = df.set_index('desig')
print('Data read')
df = df.rename(columns = {'epc':'epoch', 'meanAn': 'meanAnomaly', 'perihel':'periheleonArgument'})
df = df.rename(columns = {'nodAL':'nodeAscendingLongitude', 'incl': 'inclination'})
df = df.rename(columns = {'motDail': 'motionDailyMean', 'z':'uncertainty', 'ref': 'reference'})
df = df.rename(columns = {'nOb': 'nObservations', 'nP':'nOppositions', 'arcL':'arcLength'})
df = df.rename(columns = {'rms':'rmsResidual', 'p': 'perturbersCoarse', 'pp':'perturbersPrecise'})
df = df.rename(columns = {'comp':'computerName', 'flg':'flags', 'lastO':'lastObservationDate'})
print('Columns renamed')
print('Done')

Data read
Columns renamed
Done


In [4]:
df['lastObservationDate'] = df['lastObservationDate'].astype(str)
print('Dates stringed')
df['lastObservationDate'] = df['lastObservationDate'].apply(lambda x: x[0:-2])
print('Dates parsed')
df['lastObservationDate'] = pandas.to_datetime(df['lastObservationDate'], format='%Y%m%d', errors='coerce')
print('Dates dated')
df['lastObservationYear'] = df['lastObservationDate'].apply(lambda x: x.year)
print('Years yeared')

df['orbit_type'] = 'e'
print('Orbit type set to e')

xephem_column_order = ['name', 'orbit_type', 'inclination']
xephem_column_order += ['nodeAscendingLongitude', 'periheleonArgument', 'a']
xephem_column_order += ['motionDailyMean', 'e', 'meanAnomaly']
xephem_column_order += ['epoch', 'lastObservationYear', 'h', 'g']
xephem_df = df[xephem_column_order]
print('xephem xephemed')
print('Done')

Dates stringed
Dates parsed
Dates dated
Years yeared
Orbit type set to e
xephem xephemed
Done


In [21]:
# structure of .epb file (what we need to convert to):
# names, {f|e|h|p|E|P}, incli, long-ascending-node, arg-of-perihel, a (mean dist), 
##  mean-daily-motion, eccentricity, mean-anomaly, epoch, equinox-year, mag1, mag2, ang-size

'''
xephem_column_order = ['name', 'orbit_type', 'inclination']
xephem_column_order += ['nodeAscendingLongitude', 'periheleonArgument', 'a']
xephem_column_order += ['motionDailyMean', 'e', 'meanAnomaly']
xephem_column_order += ['epoch', 'lastObservationYear', 'h', 'g']

xephem_df = df[xephem_column_order]
'''

#aline = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
#print(aline)
#print('\n')

with open('orbits.csv', 'w') as f:
#    for i in range(0,len(df)):
    for i in range(0,101):
        if (numpy.mod(i,10) == 0):
            print('Writing data for asteroid {:}...'.format(i))
        thisline = ""
        for k in range(0,11):
            thisline += str(xephem_df.iloc[i].values[k]) + ","
        thisline += 'H '+str(xephem_df.iloc[i].values[11]) + ","
        thisline += str(xephem_df.iloc[i].values[12])
        thisline += '\n'
#        print(thisline)
        f.write(thisline)
print('Done')

Writing data for asteroid 0...
Writing data for asteroid 10...
Writing data for asteroid 20...
Writing data for asteroid 30...
Writing data for asteroid 40...
Writing data for asteroid 50...
Writing data for asteroid 60...
Writing data for asteroid 70...
Writing data for asteroid 80...
Writing data for asteroid 90...
Writing data for asteroid 100...
Done


In [22]:
ra_list = []
dec_list = []
cnt = 1
for line in open('orbits.csv'):
#    aline = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
#    print(line)
#    print(aline)
    yh = ephem.readdb(line)
    if (numpy.mod(cnt,10) == 0):
        print('Read data for asteroid {:}'.format(cnt))
    yh.compute('2017/9/1')
    if (numpy.mod(cnt,10) == 0):
        print('Calculated ra/dec for asteroid {:}'.format(cnt))
    ra_list.append(str(yh.ra))
    dec_list.append(str(yh.dec))
    cnt = cnt + 1

print('Done')

Read data for asteroid 10
Calculated ra/dec for asteroid 10
Read data for asteroid 20
Calculated ra/dec for asteroid 20
Read data for asteroid 30
Calculated ra/dec for asteroid 30
Read data for asteroid 40
Calculated ra/dec for asteroid 40
Read data for asteroid 50
Calculated ra/dec for asteroid 50
Read data for asteroid 60
Calculated ra/dec for asteroid 60
Read data for asteroid 70
Calculated ra/dec for asteroid 70
Read data for asteroid 80
Calculated ra/dec for asteroid 80
Read data for asteroid 90
Calculated ra/dec for asteroid 90
Read data for asteroid 100
Calculated ra/dec for asteroid 100
Done


In [24]:
coord_deg = numpy.empty((len(df),2))
coord_deg[:] = numpy.nan

for i in range(0, len(ra_list)):
    thestr = ra_list[i] + ' ' + dec_list[i]
#    print(thestr)
    a = SkyCoord(thestr, unit='hourangle, deg')
#    print(a)
#    print(a.ra.deg, a.dec.deg)
    coord_deg[i,0] = a.ra.deg
    coord_deg[i,1] = a.dec.deg
    if (numpy.mod(i, 10) == 0):
        print('Converted coordinates for asteroid {:}'.format(i))
#    df.loc[i, 'ra'] = a.ra.deg
#    df.loc[i, 'dec'] = a.dec.deg
#    print('df updated')
#    enya1927_df.loc[i, 'game_in_season'] = thisgame
print(coord_deg)
df['ra'] = coord_deg[:,0].tolist()
df['dec'] = coord_deg[:,1].tolist()
print('Done')
df.head(100)

ra/dec initially set to nan
Converted coordinates for asteroid 0
Converted coordinates for asteroid 10
Converted coordinates for asteroid 20
Converted coordinates for asteroid 30
Converted coordinates for asteroid 40
Converted coordinates for asteroid 50
Converted coordinates for asteroid 60
Converted coordinates for asteroid 70
Converted coordinates for asteroid 80
Converted coordinates for asteroid 90
Converted coordinates for asteroid 100
[[ 296.86983333  -31.94716667]
 [ 233.010625     13.87986111]
 [ 270.61425      -9.64502778]
 ..., 
 [          nan           nan]
 [          nan           nan]
 [          nan           nan]]
Done


,h,g,epoch,meanAnomaly,periheleonArgument,nodeAscendingLongitude,inclination,e,motionDailyMean,a,...,perturbersCoarse,perturbersPrecise,computerName,flags,name,lastObservationDate,lastObservationYear,orbit_type,ra,dec
desig,,,,,,,,,,,,,,,,,,,,,
 00001,3.34,0.12,K1794,309.49412,73.02368,80.30888,10.59322,0.075607,0.214089,2.767409,...,M-v,30h,MPCLINUX,0000,(1) Ceres,2017-03-05,2017.0,e,296.869833,-31.947167
 00002,4.13,0.11,K1794,291.65136,309.99154,173.08718,34.83792,0.230597,0.213432,2.773085,...,M-v,28h,MPCLINUX,0000,(2) Pallas,2017-06-17,2017.0,e,233.010625,13.879861
 00003,5.33,0.32,K1794,259.23491,248.20648,169.85828,12.98996,0.256853,0.226097,2.668531,...,M-v,38h,MPCLINUX,0000,(3) Juno,2017-06-15,2017.0,e,270.614250,-9.645028
 00004,3.20,0.32,K1794,292.71034,150.94297,103.83591,7.14002,0.089152,0.271547,2.361778,...,M-p,18h,MPCLINUX,0000,(4) Vesta,2017-03-23,2017.0,e,359.698292,-12.161194
 00005,6.85,0.15,K1794,139.06218,358.75689,141.58349,5.36794,0.191465,0.238766,2.573285,...,M-v,38h,MPCLINUX,0000,(5) Astraea,2017-06-15,2017.0,e,106.369042,18.796222
 00006,5.71,0.24,K1794,289.56753,239.86057,138.64807,14.73750,0.202802,0.261011,2.424916,...,M-v,38h,MPCLINUX,0007,(6) Hebe,2017-05-28,2017.0,e,22.105750,-9.430389
 00007,5.51,0.15,K1794,339.84951,145.31450,259.56448,5.52317,0.231441,0.267459,2.385783,...,M-v,38h,MPCLINUX,0000,(7) Iris,2017-06-28,2017.0,e,332.305708,-0.282750
 00008,6.49,0.28,K1794,13.76136,285.31022,110.90088,5.88654,0.157029,0.301755,2.201405,...,M-v,38h,MPCLINUX,0000,(8) Flora,2017-07-04,2017.0,e,95.960125,19.676417
 00009,6.28,0.17,K1794,116.70276,6.19939,68.91720,5.57497,0.122113,0.267215,2.387237,...,M-v,38h,MPCLINUX,0000,(9) Metis,2017-06-23,2017.0,e,169.301250,8.790194


In [ ]:
>>> line = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
>>> yh = ephem.readdb(line)
>>> yh.compute('2007/10/1')
>>> print('%.10f' % yh.earth_distance)
14.8046731949
>>> print(yh.mag)
23.96